In [35]:
import pandas as pd
import random
import yaml
import json

data = pd.read_csv("safegraph_cbg_population_estimate.csv")
display(data.head())

,census_block_group,B00001e1,B00001m1,B00002e1,B00002m1
0,10010201001,72.0,-1.0,32.0,-1.0
1,10010201002,173.0,-1.0,66.0,-1.0
2,10010202001,100.0,-1.0,43.0,-1.0
3,10010202002,167.0,-1.0,54.0,-1.0
4,10010203001,184.0,-1.0,80.0,-1.0


In [ ]:
STATE_FIPS = "26"

# Filter the data for the specified state
cbgs = list(data['census_block_group'])
states = [str(cbg)[:2] for cbg in cbgs]
state_data = [cbg for cbg, state in zip(cbgs, states) if state == STATE_FIPS]
print(len(state_data))

# print list of counties in the state
counties = set([str(cbg)[2:5] for cbg in state_data])
print(len(counties), "counties in the data")
print(counties)


8205
83 counties in the data
{'017', '093', '121', '011', '037', '095', '105', '123', '075', '001', '131', '163', '029', '087', '019', '021', '055', '039', '097', '077', '111', '051', '053', '109', '165', '155', '143', '069', '133', '081', '009', '157', '079', '151', '117', '071', '025', '137', '041', '147', '005', '059', '103', '043', '089', '139', '145', '045', '049', '141', '119', '047', '101', '085', '107', '153', '091', '057', '129', '063', '035', '135', '113', '149', '061', '161', '125', '083', '033', '065', '027', '023', '159', '013', '031', '003', '115', '067', '127', '015', '073', '099', '007'}


In [28]:
COUNTY_FIPS = ["093", "121"]

# Pick 80% of the cbgs from each county
county_data = [cbg for cbg in state_data if str(cbg)[2:5] in COUNTY_FIPS]
# shuffle the data
random.shuffle(county_data)
# pick random portion of the data  
county_data = county_data[:int(0.5*len(county_data))]
# print the number of cbgs
print(len(county_data))


125


In [40]:
# Create dictionary of cbgs in county_data and their population
cbg_pop = {cbg: data[data['census_block_group'] == cbg]['B00001e1'].values[0] for cbg in county_data}
# convert populations to integers
cbg_pop = {str(cbg): int(pop) for cbg, pop in cbg_pop.items()}
cbg_pop = {"GEOIDs": cbg_pop}

# Write the dictionary to a yaml file
with open("CZ_output_test1.yaml", "w") as f:
    yaml.dump(cbg_pop, f)